In [49]:
import pickle as pkl, pandas as pd, numpy as np, json, csv ,random, timeit, re
from datetime import date, timedelta

from shapely.geometry.polygon import Polygon
from shapely.geometry.multipolygon import MultiPolygon
from descartes import PolygonPatch

from collections import OrderedDict

import matplotlib as mpl
from matplotlib import pyplot as plt

from collections import OrderedDict

In [4]:
%run ../src/geo_utils.py
%run ../src/shared_utils

In [6]:
counties = OrderedDict()
with open("../data/raw/germany_county_shapes.json", "r") as data_file:
    shape_data = json.load(data_file)

for idx, val in enumerate(shape_data["features"]):
    id_current = val["properties"]["RKI_ID"]
    name_current = val["properties"]["RKI_NameDE"]

    counties[name_current] = id_current

In [114]:
covid19_data = pd.read_csv('../data/raw/COVID19.csv', sep=',')

start_date = pd.Timestamp(2020, 1, 28)
end_date = pd.Timestamp(2020, 3, 23)
dates = [day for day in pd.date_range(start_date, end_date)]
df = pd.DataFrame(index=dates)
for county_name in counties:
    series = np.zeros(len(df), dtype=np.int32)
    lk_data = covid19_data[covid19_data['Landkreis'] == county_name]
    for (did, day) in enumerate(dates):
        day_string = "{:04d}-{:02d}-{:02d}T00:00:00.000Z".format(day.year, day.month, day.day)
        cases = np.sum(lk_data[lk_data['Meldedatum'] == day_string]['AnzahlFall'])
        if cases != 0:
            series[did] = cases
    df.insert(len(df.columns), counties[county_name], series)

df.to_csv('../data/diseases/covid19.csv', sep=";")